일반화 성능도 있기 때문에 전체 데이터셋에 대해서 성능 평가를 해봐야 함

In [2]:
import pickle
with open('../models/BernoulliNB norm l30_r15.pkl', 'rb') as f:
    classifier = pickle.load(f)
    
from py.utils import load_data
head = 'l30_r15'
directory = '../data/'
x, y, x_words, vocabs = load_data(head, directory)

x shape = (15715, 4551)
y shape = (15715,)
# features = 4551
# L words = 15715


In [3]:
word_frequency = x.sum(axis=1)
word_frequency

matrix([[100],
        [295],
        [104],
        ..., 
        [ 31],
        [ 93],
        [199]])

In [4]:
def get_sample_l(ri, topk=5):
    nonzero = x[:,ri].nonzero()[0]
    base = min(50, len(nonzero)//2)
    return [x_words[l] for l in sorted(nonzero, key=lambda x:word_frequency[x,0], reverse=True)[base:base+topk]]

coefficient = log prob

==> exp(coefficient) = prob

In [6]:
import numpy as np
np.exp(classifier.class_log_prior_)

array([ 0.19987273,  0.80012727])

In [7]:
import numpy as np

base = 0
topk = 100
for r, coef in sorted(enumerate(classifier.coef_[0]), key=lambda x:x[1], reverse=True)[base :base + topk]:
    print('%s - %s (logp= %.3f, p= %.3f)' % (get_sample_l(r), vocabs[r], coef, np.exp(coef)))

['안', '지금', '이상', '모습', '사용'] - 의 (logp= -0.184, p= 0.832)
['어머니', '중', '이것', '여기', '사이'] - 에 (logp= -0.261, p= 0.770)
['무엇', '보이', '마음', '얼굴', '전'] - 도 (logp= -0.595, p= 0.552)
['지금', '이상', '맞', '모습', '사용'] - 을 (logp= -0.620, p= 0.538)
['선생', '사랑', '삶', '돈', '데'] - 이 (logp= -0.647, p= 0.523)
['지금', '이상', '맞', '모습', '사용'] - 은 (logp= -0.737, p= 0.479)
['안', '이상', '모습', '사용', '길'] - 에서 (logp= -0.830, p= 0.436)
['세상', '현실', '과정', '학생', '글'] - 과 (logp= -0.847, p= 0.429)
['변화', '글', '존재', '누구', '시'] - 를 (logp= -0.941, p= 0.390)
['시대', '문화', '친구', '어디', '개'] - 가 (logp= -0.951, p= 0.387)
['죽', '사이', '사', '맞', '나오'] - 는 (logp= -1.020, p= 0.360)
['이름', '여성', '한국', '세상', '과정'] - 으로 (logp= -1.041, p= 0.353)
['어디', '현실', '개', '변화', '글'] - 로 (logp= -1.064, p= 0.345)
['존재', '누구', '시', '저', '자체'] - 와 (logp= -1.125, p= 0.325)
['죽', '사이', '안', '지금', '이상'] - 이다 (logp= -1.226, p= 0.293)
['모습', '길', '다음', '세계', '자기'] - 인 (logp= -1.291, p= 0.275)
['내용', '책', '힘', '뿐', '뜻'] - 이나 (logp= -1.305, p= 0.271)
['몸

## BernoulliNB classifier code

    def _joint_log_likelihood(self, X):
        """Calculate the posterior log probability of the samples X"""
        check_is_fitted(self, "classes_")

        X = check_array(X, accept_sparse='csr')

        if self.binarize is not None:
            X = binarize(X, threshold=self.binarize)

        n_classes, n_features = self.feature_log_prob_.shape
        n_samples, n_features_X = X.shape

        if n_features_X != n_features:
            raise ValueError("Expected input with %d features, got %d instead"
                             % (n_features, n_features_X))

        neg_prob = np.log(1 - np.exp(self.feature_log_prob_))
        # Compute  neg_prob · (1 - X).T  as  ∑neg_prob - X · neg_prob
        jll = safe_sparse_dot(X, (self.feature_log_prob_ - neg_prob).T)
        jll += self.class_log_prior_ + neg_prob.sum(axis=1)

        return jll

### Bernoulli Naive Bayes 역시 infrequent feature가 low scale

classification rule: $ P(x_i \mid y) = P(i \mid y) x_i + (1 - P(i \mid y)) (1 - x_i) $

class y에서 feature i의 생성확률을 만들기 때문에 infrequent 한 feature가 작은 coefficient를 지님

In [10]:
np.exp(classifier.coef_[0,vocabs.index('해줌으로써')])

0.0011132315521628507